In [6]:
import sys
sys.path.append('/Users/ygtong/research/vlasov_solver/')

In [7]:
import numpy as np
import scipy.optimize
from py_vlasov.util import zp, kzkp, list_to_complex, real_imag, do_cprofile
from py_vlasov.wrapper import oblique_wrapper, disp_det, input_gen
from py_vlasov.transport_ratios import transport_ratios
from py_vlasov.follow_parameter import follow_k
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.minor.size'] = 3.5
mpl.rcParams['ytick.minor.size'] = 3.5
mpl.rcParams['xtick.major.width'] = 1.1
mpl.rcParams['ytick.major.width'] = 1.1
mpl.rcParams['xtick.minor.width'] = .8
mpl.rcParams['ytick.minor.width'] = .8
mpl.rcParams['legend.fontsize'] = 14
#xtick.major.size     : 3.5      # major tick size in points
#xtick.minor.size     : 2      # minor tick size in points
#xtick.major.width    : 0.8    # major tick width in points
#xtick.minor.width    : 0.6    # minor tick width in points
%matplotlib inline

In [8]:
def do_cprofile(func):
    """
    wrapper of func to do profile 
    """

    import cProfile, pstats
    
    def profiled_func(*args, **kwargs):
        profile = cProfile.Profile()
        try:
            profile.enable()
            result = func(*args, **kwargs)
            profile.disable()
            return result
        finally:
            ps = pstats.Stats(profile)
            ps.strip_dirs().sort_stats('time').print_stats(20)
    return profiled_func

In [9]:
aol = np.sqrt(2.65e-8)
print('aol = ', aol)
tc_tp = 2
th_tc = 6
nc_np = 0.95
nh_np = 1 - nc_np
vc_va = 2.42
vh_va = -vc_va * nc_np / nh_np
print('vh_va = {0:.3g}'.format(vh_va))

betap = 0.25
theta = 85


t_list=[1., tc_tp, th_tc * tc_tp]
a_list=[1., 1., 1.]
n_list=[1., nc_np, nh_np] 
q_list=[1.,-1., -1.]
m_list=[1., 1./1836., 1./1836]
v_list=[0., vc_va, vh_va]
n = 6
method = 'numpy'

k = 0.2
kz, kp = kzkp(k, theta)

param = [kz, kp, betap, t_list, a_list, n_list, q_list,
         m_list, v_list, n, method, aol]

aol =  0.000162788205961
vh_va = -46


In [10]:
seed_freq = kz / np.sqrt(betap)
f = lambda wrel:\
    real_imag(oblique_wrapper(list_to_complex(wrel), kz, kp, betap,
                              t_list, a_list, n_list, q_list, \
                              m_list, v_list, method = method, n = n, \
                              aol=aol))
guess = seed_freq 
print('seed_freq = {0:.3g}'.format(guess))

# wr, wi = scipy.optimize.fsolve(f, real_imag(guess))
wr, wi = scipy.optimize.root(f, real_imag(guess))['x']

print(wr)
print(wi)

seed_freq = 0.0349
0.0345575003836
-2.27067334544e-05


In [11]:
scipy.optimize.root(f, real_imag(guess))['x']

array([  3.45575004e-02,  -2.27067335e-05])

In [12]:
@do_cprofile
def find_w():
    seed_freq = kz / np.sqrt(betap)
    f = lambda wrel:\
        real_imag(oblique_wrapper(list_to_complex(wrel), kz, kp, betap,
                                  t_list, a_list, n_list, q_list, \
                                  m_list, v_list, method = method, n = n, \
                                  aol=aol))
    guess = seed_freq 
    print('seed_freq = {0:.3g}'.format(guess))

    wr, wi = scipy.optimize.fsolve(f, real_imag(guess))
    print(wr)
    print(wi)
    return

@do_cprofile
def find_w():
    seed_freq = kz / np.sqrt(betap)
    f = lambda wrel:\
        real_imag(oblique_wrapper(list_to_complex(wrel), kz, kp, betap,
                                  t_list, a_list, n_list, q_list, \
                                  m_list, v_list, method = method, n = n, \
                                  aol=aol))
    guess = seed_freq 
    print('seed_freq = {0:.3g}'.format(guess))

    wr, wi = scipy.optimize.fsolve(f, real_imag(guess))
    print(wr)
    print(wi)
    return

In [14]:
find_w()

seed_freq = 0.0349
0.0345575003836
-2.27067334544e-05
         2106 function calls in 0.021 seconds

   Ordered by: internal time
   List reduced from 70 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       30    0.004    0.000    0.017    0.001 dispersion_tensor.py:176(f_yn_arr)
       30    0.004    0.000    0.004    0.000 dispersion_tensor.py:200(<listcomp>)
       30    0.002    0.000    0.005    0.000 dispersion_tensor.py:81(f_abn_arr)
       30    0.002    0.000    0.002    0.000 dispersion_tensor.py:199(<listcomp>)
      390    0.001    0.000    0.001    0.000 util.py:53(zp)
      200    0.001    0.000    0.001    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       30    0.001    0.000    0.018    0.001 dispersion_tensor.py:243(f_chi_vec)
       30    0.001    0.000    0.001    0.000 dispersion_tensor.py:26(f_zeta_arr)
      190    0.001    0.000    0.002    0.000 fromnumeric.py:1730(sum)
      122    0.001    0.000   